In [3]:
import numpy as np
import time
from tqdm import tqdm

In [4]:
# 首先load data
def loadData(file_name):
    data_arr = []
    label_arr = []
    
    fr = open(file_name)
    for line in fr.readlines():
        cur_line = line.strip().split(',')
        #此外将数据进行了二值化处理，大于128的转换成1，小于的转换成0，方便后续计算
        data_arr.append([int(int(num) > 128) for num in cur_line[1:]])
        label_arr.append(int(cur_line[0]))
        
    return data_arr, label_arr

In [15]:
# 然后计算先验概率和条件概率
def getAllProbability(train_data, train_label):
    feature_num = 784
    class_num = 10
    
    # 计算先验概率，并转化为log对数形式，方便计算
    py = np.zeros((class_num, 1))
    for i in range(class_num):
        py[i] = ((np.sum(np.mat(train_label) == i)) + 1) / (len(train_label) + 10)
    py = np.log(py)
    
    # 计算条件概率, px_y=P（X=x|Y=y）
    # 一共有 feature_num * class_num 个概率需要计算
    px_y = np.zeros((class_num, feature_num, 2))
    for i in range(len(train_label)): # 遍历所有样本
        label = train_label[i]
        x = train_data[i]
        for j in range(feature_num): # 遍历每一个feature
            # 之前做了预处理，feature不是0就是1
            px_y[label][j][x[j]] += 1
            
    for label in range(class_num):
        for j in range(feature_num):
            px_y0 = px_y[label][j][0]
            px_y1 = px_y[label][j][1]
            
            # 将概率转换为log对数，这个概率表示 y=label时，第j个feature取各种值的概率
            px_y[label][j][0] = np.log((px_y0 + 1) / (px_y0 + px_y1 + 2))
            px_y[label][j][1] = np.log((px_y1 + 1) / (px_y0 + px_y1 + 2))
    
    return py, px_y

In [16]:
def NaiveBayes(py, px_y, x):
    '''
    py: 先验概率 P(Y)
    px_y: 条件概率 P(X|Y)
    x: 要估计的样本
    '''
    
    feature_num = 784
    class_num = 10
    P = [0] * class_num
    # 对于每一个类别，单独估计其概率
    for i in range(class_num):
        sum = 0
        # 对于每一个特征，获取其条件概率
        for j in range(feature_num):
            # 本来是相乘，但是转换为log后就变成相加了
            sum += px_y[i][j][x[j]]
        P[i] = sum + py[i]
    
    return P.index(max(P))  # 取argmax，返回预测的类别

In [17]:
def model_test(py, px_y, test_data, test_label):
    error_cnt = 0
    for i in tqdm(range(len(test_data))):
        predict = NaiveBayes(py, px_y, test_data[i])
        
        if predict != test_label[i]:
            error_cnt += 1
    return 1 - (error_cnt / len(test_data))

In [8]:
train_data, train_label = loadData('../../data/Mnist/train.csv')
test_data, test_label = loadData('../../data/Mnist/test.csv')

In [18]:
py, px_y = getAllProbability(train_data, train_label)

In [19]:
test_acc = model_test(py, px_y, test_data, test_label)
print(test_acc)


 38%|███▊      | 3757/10000 [00:14<00:25, 244.08it/s]


 75%|███████▍  | 7484/10000 [00:29<00:10, 247.23it/s]


100%|██████████| 10000/10000 [00:39<00:00, 251.90it/s]

0.8432999999999999


In [20]:
train_acc = model_test(py, px_y, train_data, train_label)
print(train_acc)


  6%|▋         | 3769/60000 [00:14<03:44, 250.37it/s]


 13%|█▎        | 7609/60000 [00:29<03:26, 253.48it/s]


 19%|█▉        | 11348/60000 [00:44<03:12, 252.96it/s]


 25%|██▌       | 15008/60000 [00:58<03:01, 247.76it/s]


 31%|███       | 18655/60000 [01:13<02:48, 245.53it/s]


...

 85%|████████▌ | 51025/60000 [03:23<00:35, 254.08it/s]


 91%|█████████ | 54659/60000 [03:38<00:21, 248.07it/s]


 97%|█████████▋| 58258/60000 [03:52<00:06, 252.77it/s]


100%|██████████| 60000/60000 [03:59<00:00, 250.12it/s]

0.8362666666666667
